# ***Librariess***


In [1]:
import pandas as pd
import spacy
import nltk
import re
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
import gensim.downloader as api






In [2]:
nlp_urdu = spacy.blank("ur")
nltk.download('wordnet')
nltk.download('omw')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

# ***LOAD DATA SET***

In [3]:
def load_urdu_sentiment_data(filename):
    tweets = []
    labels = []

    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip().split('\t')
            if len(line) == 2:
                tweets.append(line[0])
                labels.append(line[1])

    return tweets, labels

In [4]:
def read_conllu_file(file_path):
    words_roots = []
    with open(file_path, 'r', encoding='utf-8') as conllufile:
        lines = conllufile.readlines()
        for line in lines:
            if line.strip() and not line.startswith("#"):
                parts = line.split('\t')
                if len(parts) >= 3:
                    word, root = parts[1], parts[2]
                    words_roots.append((word, root))
    return words_roots


# ***Words AND ROOTS***

In [5]:

file_path = 'sample.conllu'
words_roots = read_conllu_file(file_path)
print(len(words_roots))
for word, root in words_roots:
    print("Word:", word, "\tRoot:", root)


3178
Word: صدر 	Root: صدر
Word: تلگودیشم 	Root: تلگودیشم
Word: مسٹر 	Root: مسٹر
Word: این 	Root: این
Word: چندرا 	Root: چندرا
Word: بابو 	Root: بابو
Word: نائیڈو 	Root: نائیڈو
Word: نے 	Root: نے
Word: کڑپہ 	Root: کڑپہ
Word: کے 	Root: کا
Word: ضمنی 	Root: ضمنی
Word: انتخابات 	Root: انتخاب
Word: مےں 	Root: مےں
Word: اضافی 	Root: اضافی
Word: فورسیس 	Root: فورسی
Word: کی 	Root: کا
Word: تعیناتی 	Root: تعیناتی
Word: کا 	Root: کا
Word: مطالبہ 	Root: مطالبہ
Word: کرتے 	Root: کر
Word: ہوئے 	Root: ہو
Word: چیف 	Root: چیف
Word: الیکشن 	Root: الیکشن
Word: کمشنر 	Root: کمشنر
Word: کو 	Root: کو
Word: مکتوب 	Root: مکتوب
Word: روانہ 	Root: روانہ
Word: کیا 	Root: کر
Word: ۔ 	Root: ۔
Word: انہوں 	Root: وہ
Word: نے 	Root: نے
Word: کڑپہ 	Root: کڑپہ
Word: کے 	Root: کا
Word: پارلیمانی 	Root: پارلیمانی
Word: اور 	Root: اور
Word: پولی 	Root: پولی
Word: ویندلہ 	Root: ویندلہ
Word: کے 	Root: کا
Word: اسمبلی 	Root: اسمبلی
Word: انتخابات 	Root: انتخاب
Word: کو 	Root: کو
Word: صاف 	Root: صاف
Word: و 	Root: و
Word:

# ***DATA***

In [6]:
filename = 'urdu-sentiment-corpus-v1.tsv'
tweets, labels = load_urdu_sentiment_data(filename)
preprocessed_tweets = []
for tweet in tweets:
    preprocessed_tweet = re.sub(r'[؛۔٫٪+=@#!؟،۔)(}{]', '', tweet)
    preprocessed_tweets.append(preprocessed_tweet)
data = {'Tweet': preprocessed_tweets, 'Label': labels}
data = pd.DataFrame(data)
data

,Tweet,Label
0,Tweet,Class
1,میں نے ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ ...,P
2,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
3,ٹویٹر کا خیال کیسے آیا,O
4,"سرچ انجن گوگل کے نائب صدر نے فضا میں 130,000 ...",P
...,...,...
995,اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے,P
996,چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...,P
997,واہ جناب واہ اچھی رہی جناب خود کو فرشتہ سمجوں ...,P
998,اسلام آباد :پی اے ٹی کا دھرنا ختم صفائی کے کام...,P


# ***`---------------PRE PROCESSING--------------------------`***

# ***Remove Digits***

In [7]:
urdu_digits = ['۶', '۴', '۵', '۸', '۲', '۰', '۷', '۹', '۳', '۱']

english_digits=['1','2','3','4','5','6','7','8','9','0']

def remove_numbers(text):
    pattern = '[' + ''.join(urdu_digits) + ''.join(english_digits) + ']'
    text = re.sub(pattern, '', text)
    return text

In [8]:
data['Tweet'] = data['Tweet'].apply(lambda x: remove_numbers(x))
data

,Tweet,Label
0,Tweet,Class
1,میں نے ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ ...,P
2,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
3,ٹویٹر کا خیال کیسے آیا,O
4,"سرچ انجن گوگل کے نائب صدر نے فضا میں , فٹ کی ...",P
...,...,...
995,اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے,P
996,چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...,P
997,واہ جناب واہ اچھی رہی جناب خود کو فرشتہ سمجوں ...,P
998,اسلام آباد :پی اے ٹی کا دھرنا ختم صفائی کے کام...,P


# ***TOKENIZE***

In [9]:
def tokenize_with_spacy(data):
    nlp = spacy.blank("ur")
    tokenized_data = []
    for tweet, label in zip(data['Tweet'], data['Label']):
        doc = nlp(tweet)
        tokenized_tweet = [(token.text, label) for token in doc]
        tokenized_data.extend(tokenized_tweet)
    return tokenized_data


In [10]:
tokenized_data = tokenize_with_spacy(data)
print("Total NUmber of Words ",len(tokenized_data))
tokenized_data

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Total NUmber of Words  17639


[('Tweet', 'Class'),
 ('میں', 'P'),
 ('نے', 'P'),
 ('ایٹم', 'P'),
 ('بم', 'P'),
 ('بنایا', 'P'),
 ('ھے', 'P'),
 ('او', 'P'),
 ('بھائی', 'P'),
 ('ایٹم', 'P'),
 ('بمب', 'P'),
 ('کوٹ', 'P'),
 ('لکھپت', 'P'),
 ('والی', 'P'),
 ('اتفاق', 'P'),
 ('فیکٹری', 'P'),
 ('میں', 'P'),
 ('نہیں', 'P'),
 ('بنتاایٹم', 'P'),
 ('بم', 'P'),
 ('کہوٹہ', 'P'),
 ('کی', 'P'),
 ('ایٹمی', 'P'),
 ('چندے', 'N'),
 ('سے', 'N'),
 ('انقلاب', 'N'),
 ('اور', 'N'),
 ('عمران', 'N'),
 ('خان', 'N'),
 ('وزیر', 'N'),
 ('اعظم', 'N'),
 ('نہیں', 'N'),
 ('بن', 'N'),
 ('سکتے', 'N'),
 ('ٹویٹر', 'O'),
 ('کا', 'O'),
 ('خیال', 'O'),
 ('کیسے', 'O'),
 ('آیا', 'O'),
 ('سرچ', 'P'),
 ('انجن', 'P'),
 ('گوگل', 'P'),
 ('کے', 'P'),
 ('نائب', 'P'),
 ('صدر', 'P'),
 ('نے', 'P'),
 ('فضا', 'P'),
 ('میں', 'P'),
 (' ', 'P'),
 (',', 'P'),
 ('فٹ', 'P'),
 ('کی', 'P'),
 ('بلندی', 'P'),
 ('پر', 'P'),
 ('چھلانگ', 'P'),
 ('لگا', 'P'),
 ('کر', 'P'),
 ('عالمی', 'P'),
 ('ریکارڈ', 'P'),
 ('قائم', 'P'),
 ('کرلیا', 'P'),
 ('چھلانگ', 'P'),
 ('کی', 'P'),
 ('ابھی', 'P

# ***REMOVE STOP WODS***

In [11]:
def remove_stopwords_spacy_urdu(data):
    words = [word for word, _ in data]
    labels = [label for _, label in data]
    doc = nlp_urdu(" ".join(words))
    filtered_words = [token.text for token in doc if not token.is_stop and token.is_alpha]
    filtered_labels = [label for i, label in enumerate(labels) if i < len(doc) and not doc[i].is_stop and doc[i].is_alpha]
    merged_data = [(word, label) for word, label in zip(filtered_words, filtered_labels)]

    return merged_data


In [12]:
filtered_words = remove_stopwords_spacy_urdu(tokenized_data)
print("Total Words After removing Stop Words ",len(filtered_words))
filtered_words

Total Words After removing Stop Words  13592


[('Tweet', 'Class'),
 ('میں', 'P'),
 ('نے', 'P'),
 ('ایٹم', 'P'),
 ('بم', 'P'),
 ('بنایا', 'P'),
 ('ھے', 'P'),
 ('او', 'P'),
 ('بھائی', 'P'),
 ('ایٹم', 'P'),
 ('بمب', 'P'),
 ('کوٹ', 'P'),
 ('لکھپت', 'P'),
 ('والی', 'P'),
 ('اتفاق', 'P'),
 ('فیکٹری', 'P'),
 ('میں', 'P'),
 ('نہیں', 'P'),
 ('بنتاایٹم', 'P'),
 ('بم', 'P'),
 ('کہوٹہ', 'P'),
 ('ایٹمی', 'P'),
 ('چندے', 'N'),
 ('سے', 'N'),
 ('انقلاب', 'N'),
 ('عمران', 'N'),
 ('خان', 'N'),
 ('وزیر', 'N'),
 ('اعظم', 'N'),
 ('نہیں', 'N'),
 ('بن', 'N'),
 ('سکتے', 'N'),
 ('ٹویٹر', 'O'),
 ('کا', 'O'),
 ('خیال', 'O'),
 ('کیسے', 'O'),
 ('آیا', 'O'),
 ('سرچ', 'P'),
 ('انجن', 'P'),
 ('گوگل', 'P'),
 ('نائب', 'P'),
 ('صدر', 'P'),
 ('نے', 'P'),
 ('فضا', 'P'),
 ('میں', 'P'),
 ('فٹ', 'P'),
 ('بلندی', 'P'),
 ('چھلانگ', 'P'),
 ('لگا', 'P'),
 ('عالمی', 'P'),
 ('ریکارڈ', 'P'),
 ('قائم', 'P'),
 ('کرلیا', 'P'),
 ('چھلانگ', 'P'),
 ('ابھی', 'P'),
 ('اسکی', 'P'),
 ('لہریں', 'P'),
 ('کبھی', 'P'),
 ('کبھی', 'P'),
 ('آ', 'P'),
 ('جاتی', 'P'),
 ('یار', 'P'),
 ('گندی', 'N

# ***LEMATIZATION***

In [13]:
def urdu_lemmatize_with_roots(words, words_roots):
    headers = words[0]
    words_data = words[1:]
    lemmatized_words = []
    for word_tuple in words_data:
        word = word_tuple[0]
        found_root = False
        for w, root in words_roots:
            if word == w:
                lemmatized_words.append((root, word_tuple[1]))
                found_root = True
                break
        if not found_root:
            lemmatized_words.append(word_tuple)
    merged = [('Tweet', 'Class')] + lemmatized_words
    return merged


In [14]:

lemmatized_words = urdu_lemmatize_with_roots(filtered_words, words_roots)
print("Original Words:", filtered_words)
print("Lemmatized Words:", lemmatized_words)
lemmatized_df = pd.DataFrame(lemmatized_words, columns=['Tweet', 'Class'])



Original Words: [('Tweet', 'Class'), ('میں', 'P'), ('نے', 'P'), ('ایٹم', 'P'), ('بم', 'P'), ('بنایا', 'P'), ('ھے', 'P'), ('او', 'P'), ('بھائی', 'P'), ('ایٹم', 'P'), ('بمب', 'P'), ('کوٹ', 'P'), ('لکھپت', 'P'), ('والی', 'P'), ('اتفاق', 'P'), ('فیکٹری', 'P'), ('میں', 'P'), ('نہیں', 'P'), ('بنتاایٹم', 'P'), ('بم', 'P'), ('کہوٹہ', 'P'), ('ایٹمی', 'P'), ('چندے', 'N'), ('سے', 'N'), ('انقلاب', 'N'), ('عمران', 'N'), ('خان', 'N'), ('وزیر', 'N'), ('اعظم', 'N'), ('نہیں', 'N'), ('بن', 'N'), ('سکتے', 'N'), ('ٹویٹر', 'O'), ('کا', 'O'), ('خیال', 'O'), ('کیسے', 'O'), ('آیا', 'O'), ('سرچ', 'P'), ('انجن', 'P'), ('گوگل', 'P'), ('نائب', 'P'), ('صدر', 'P'), ('نے', 'P'), ('فضا', 'P'), ('میں', 'P'), ('فٹ', 'P'), ('بلندی', 'P'), ('چھلانگ', 'P'), ('لگا', 'P'), ('عالمی', 'P'), ('ریکارڈ', 'P'), ('قائم', 'P'), ('کرلیا', 'P'), ('چھلانگ', 'P'), ('ابھی', 'P'), ('اسکی', 'P'), ('لہریں', 'P'), ('کبھی', 'P'), ('کبھی', 'P'), ('آ', 'P'), ('جاتی', 'P'), ('یار', 'P'), ('گندی', 'N'), ('زبان', 'N'), ('گٹر', 'N'), ('جیسے', 'N')

In [15]:
lemmatized_df = lemmatized_df.iloc[1:]
lemmatized_df

,Tweet,Class
1,میں,P
2,نے,P
3,ایٹم,P
4,بم,P
5,بنا,P
...,...,...
13587,ساتھتم,P
13588,بھی,P
13589,یوں,P
13590,جب,P


# ***-------------------------------MODELS FORMATION--------------------------------***

# ***`SPLIT DATA`***

In [16]:
lemmatized_df = lemmatized_df.dropna(subset=['Class'])
binary_labels = {'P': 1, 'N': 0, 'O': 0}
lemmatized_df['Class'] = lemmatized_df['Class'].map(binary_labels)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lemmatized_df['Tweet'])
X = tokenizer.texts_to_sequences(lemmatized_df['Tweet'])
max_sequence_length = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_sequence_length)
y = lemmatized_df['Class'].values


In [17]:
results = {
    'BILSTM (without embeddings)': {'F-score': 0.6 , 'Accuracy': 0.52 , 'Precision': 0.22 , 'Recall': 0.47 },
}

In [18]:
table = pd.DataFrame.from_dict(results, orient='index')
print(table)

                             F-score  Accuracy  Precision  Recall
BILSTM (without embeddings)      0.6      0.52       0.22    0.47


# ***BUILT BILSTM MODEL***

In [19]:
def build_bilstm_model(input_shape, embedding_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix], trainable=False, input_shape=input_shape))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# ***Embeding MEtrix***

In [20]:
def get_embedding_matrix_word2vec(model):
    embedding_dim = model.vector_size
    embedding_matrix = np.zeros((len(model.index_to_key), embedding_dim))
    for i, word in enumerate(model.index_to_key):
        embedding_matrix[i] = model[word]
    return embedding_matrix


In [21]:
def elmo_embeddings(text):
    embeddings = elmo_m0del(text, signature="default", as_dict=True)["elmo"]
    return tf.reduce_mean(embeddings, axis=1)


# ***`-------WORD2VEC-------------------------`***

In [68]:
word2vec_model = api.load("word2vec-google-news-300")


In [69]:
max_index = np.max(X)
embedding_matrix = np.zeros((max_index + 1, 300))
for i in range(max_index + 1):
    if str(i) in word2vec_model:
        embedding_matrix[i] = word2vec_model[str(i)]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [71]:
input_shape = (X_train.shape[1],)

# Build the BiLSTM model
model = build_bilstm_model(input_shape, embedding_matrix)

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
144/144 [==============================] - 11s 22ms/step - loss: 0.6927 - accuracy: 0.5189 - val_loss: 0.6937 - val_accuracy: 0.4951
Epoch 2/5
144/144 [==============================] - 1s 10ms/step - loss: 0.6925 - accuracy: 0.5215 - val_loss: 0.6938 - val_accuracy: 0.5029
Epoch 3/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6922 - accuracy: 0.5225 - val_loss: 0.6945 - val_accuracy: 0.4980
Epoch 4/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6922 - accuracy: 0.5213 - val_loss: 0.6938 - val_accuracy: 0.4990
Epoch 5/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6920 - accuracy: 0.5227 - val_loss: 0.6949 - val_accuracy: 0.4990


In [72]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results['BILSTM with word2vec'] = {'F-score': f1, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall}

107/107 [==============================] - 2s 4ms/step


RESULTS

In [73]:
table = pd.DataFrame.from_dict(results, orient='index')
print(table)

                             F-score  Accuracy  Precision    Recall
BILSTM (without embeddings)  0.60000  0.520000      0.220  0.470000
BILSTM with word2vec         0.09513  0.529723      0.525  0.052304


# ***------------------------GLOVE-----------------------

```
# ------------------------GLOVE-----------------------
```

***

In [22]:
glove_model = api.load("glove-twitter-100")

In [23]:
embedding_matrix = get_embedding_matrix_word2vec(glove_model)



In [26]:
input_shape = (X_train.shape[1],)
model = build_bilstm_model(input_shape, embedding_matrix)

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results['BILSTM with glove'] = {'F-score': f1, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall}

Epoch 1/5
144/144 [==============================] - 16s 23ms/step - loss: 0.6932 - accuracy: 0.5061 - val_loss: 0.6935 - val_accuracy: 0.5078
Epoch 2/5
144/144 [==============================] - 1s 10ms/step - loss: 0.6899 - accuracy: 0.5305 - val_loss: 0.6927 - val_accuracy: 0.4971
Epoch 3/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6863 - accuracy: 0.5461 - val_loss: 0.6936 - val_accuracy: 0.5147
Epoch 4/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6833 - accuracy: 0.5497 - val_loss: 0.6938 - val_accuracy: 0.5078
Epoch 5/5
107/107 [==============================] - 2s 4ms/step


In [27]:
table = pd.DataFrame.from_dict(results, orient='index')
print(table)

                             F-score  Accuracy  Precision    Recall
BILSTM (without embeddings)  0.60000  0.520000   0.220000  0.470000
BILSTM with glove            0.44475  0.525309   0.497306  0.402242


# ***---------------------------fasttext_model------------------- ***

In [28]:
fasttext_model = api.load("fasttext-wiki-news-subwords-300")

In [29]:
embedding_matrix = get_embedding_matrix_word2vec(fasttext_model)

In [30]:
input_shape = (X_train.shape[1],)
model = build_bilstm_model(input_shape, embedding_matrix)

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results['BILSTM with fasttext'] = {'F-score': f1, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall}

Epoch 1/5
144/144 [==============================] - 11s 20ms/step - loss: 0.6925 - accuracy: 0.5192 - val_loss: 0.6923 - val_accuracy: 0.5039
Epoch 2/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6892 - accuracy: 0.5386 - val_loss: 0.6933 - val_accuracy: 0.5098
Epoch 3/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6847 - accuracy: 0.5550 - val_loss: 0.6947 - val_accuracy: 0.4980
Epoch 4/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6805 - accuracy: 0.5621 - val_loss: 0.6984 - val_accuracy: 0.5098
Epoch 5/5
107/107 [==============================] - 2s 4ms/step


In [31]:
table = pd.DataFrame.from_dict(results, orient='index')
print(table)

                              F-score  Accuracy  Precision    Recall
BILSTM (without embeddings)  0.600000  0.520000   0.220000  0.470000
BILSTM with glove            0.444750  0.525309   0.497306  0.402242
BILSTM with fasttext         0.471218  0.535021   0.509407  0.438356


# ***--------------ELMO-----------------------***

In [32]:
elmo_model = hub.load("https://tfhub.dev/google/elmo/3")
print(type(elmo_model))

<class 'tensorflow.python.trackable.autotrackable.AutoTrackable'>


In [ ]:

X_train_elmo = elmo_embeddings(X_train)
X_test_elmo = elmo_embeddings(X_test)


In [33]:
input_shape = (X_train.shape[1],)
model = build_bilstm_model(input_shape, embedding_matrix)

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
results['BILSTM with ELMO'] = {'F-score': f1, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall}

Epoch 1/5
144/144 [==============================] - 11s 20ms/step - loss: 0.6924 - accuracy: 0.5191 - val_loss: 0.6923 - val_accuracy: 0.5284
Epoch 2/5
144/144 [==============================] - 2s 11ms/step - loss: 0.6891 - accuracy: 0.5471 - val_loss: 0.6950 - val_accuracy: 0.5137
Epoch 3/5
144/144 [==============================] - 2s 13ms/step - loss: 0.6845 - accuracy: 0.5551 - val_loss: 0.6931 - val_accuracy: 0.5186
Epoch 4/5
144/144 [==============================] - 1s 9ms/step - loss: 0.6805 - accuracy: 0.5567 - val_loss: 0.6962 - val_accuracy: 0.5137
Epoch 5/5
107/107 [==============================] - 2s 4ms/step


# ***`Results`***

In [34]:
table = pd.DataFrame.from_dict(results, orient='index')
print(table)

                              F-score  Accuracy  Precision    Recall
BILSTM (without embeddings)  0.600000  0.520000   0.220000  0.470000
BILSTM with glove            0.444750  0.525309   0.497306  0.402242
BILSTM with fasttext         0.471218  0.535021   0.509407  0.438356
BILSTM with ELMO             0.404367  0.534432   0.511429  0.334371
